In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize
%matplotlib inline

In [4]:
# Load Data
cwd = os.getcwd()
# Folder Locations
train_paths = ["/DataSet/Train/Cat"
               ,"/DataSet/Train/Dog"]
test_paths = ["/DataSet/Test/Cat"
              ,"/DataSet/Test/Dog"]
categories = ['Cat', 'Dog']
# Config
imgsize = [128,128]
data_name = "custom_data"

In [5]:
#Path Array
nclass = 2

for i in range(nclass):
    path = cwd + "/" + train_paths[i]
    flist = os.listdir(path)
    print(np.array(flist).shape)
    
for i in range(nclass):
    path = cwd + "/" + test_paths[i]
    flist = os.listdir(path)
    print(np.array(flist).shape)

(500,)
(500,)
(50,)
(50,)


In [6]:
#Create Train Image Array
train_cnt = 0

for i in range(nclass):
    path = cwd + "/" + train_paths[i]
    flist = os.listdir(path)
    for f in flist:
        fullpath = os.path.join(path, f)
        #Load Image to gray scale
        currimg = imread(fullpath, flatten="gray")
        #reshape and rgb float
        graysmall = imresize(currimg, [imgsize[0], imgsize[1]])/255. 
        grayvec = np.reshape(graysmall, (1, -1))
        #Train Dataset
        if train_cnt is 0:
            trainimg = grayvec
        else:
            trainimg = np.concatenate((trainimg, grayvec), axis=0)
        train_cnt = train_cnt + 1

print ("Total %d Train images loaded." % (train_cnt))

Total 1000 Train images loaded.


In [7]:
#Create Train Label
trainlabel = np.array([1, 0]).reshape((1,2))

#Cat Label is [1 0]
for i in range(499):
    label = np.array([1, 0]).reshape((1,2))
    trainlabel = np.concatenate((trainlabel, label), axis=0)
    
#Dog Label is [0 1]
for i in range(500):
    label = np.array([0, 1]).reshape((1,2))
    trainlabel = np.concatenate((trainlabel, label), axis=0)

print(" type of trainimg %s"%(type(trainimg)))
print(" type of trainlabel %s" %(type(trainlabel)))
print(trainimg.shape)
print(trainlabel.shape)

 type of trainimg <class 'numpy.ndarray'>
 type of trainlabel <class 'numpy.ndarray'>
(1000, 16384)
(1000, 2)


In [8]:
#Create Test Image Array
test_cnt = 0

for i in range(nclass):
    path = cwd + test_paths[i]
    flist = os.listdir(path)
    for f in flist:
        fullpath = os.path.join(path, f)
        currimg = imread(fullpath, flatten="gray")
        
        #reshape and rgb float
        graysmall = imresize(currimg, [imgsize[0], imgsize[1]])/255. 
        grayvec = np.reshape(graysmall, (1, -1))
        #Test Dataset
        if test_cnt is 0:
            testimg = grayvec
        else:
            testimg = np.concatenate((testimg, grayvec), axis = 0)
        test_cnt = test_cnt + 1

print("Total %d Test images loaded."%(test_cnt))

Total 100 Test images loaded.


In [9]:
#Create Test Label
testlabel = np.array([1, 0]).reshape((1,2))
# Cat
for i in range(49):
    label = np.array([1, 0]).reshape((1,2))
    testlabel = np.concatenate((testlabel, label), axis=0)
# Dog
for i in range(50):
    label = np.array([0, 1]).reshape((1,2))
    testlabel = np.concatenate((testlabel, label), axis=0)
    
print(testimg.shape)
print(testlabel.shape)

(100, 16384)
(100, 2)


In [15]:
# NETWORK TOPOLOGIES
n_input    = imgsize[0]*imgsize[1]
n_channel  = 64
n_output   = 2

# INPUTS AND OUTPUTS
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_output])
    
# Weights (width, height, channel, number)
weights = {
    'conv_w': tf.Variable(tf.random_normal([3,3,1,n_channel], stddev=0.1)),
    'dense_w': tf.Variable(tf.random_normal([64*64*n_channel, n_output], stddev=0.1))
}

# Biases
biases = {
    'conv_b': tf.Variable(tf.random_normal([n_channel], stddev=0.1)),
    'dense_b': tf.Variable(tf.random_normal([n_output], stddev=0.1))
}

print ("NETWORK READY")

NETWORK READY


In [16]:
# MODEL
def CNN(_input, _w, _b):
    # RESHAPE
    _input_r = tf.reshape(_input, shape=[-1, imgsize[0], imgsize[1], 1])
    # CONVOLUTION
    _conv = tf.nn.conv2d(_input_r, _w['conv_w'], strides=[1, 1, 1, 1], padding='SAME')
    # ADD BIAS
    _bias = tf.nn.bias_add(_conv, _b['conv_b'])
    # RELU
    _relu = tf.nn.relu(_bias)
    # MAX-POOL
    _pool  = tf.nn.max_pool(_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # VECTORIZE
    _dense = tf.reshape(_pool, [-1, _w['dense_w'].get_shape().as_list()[0]])
    # DENSE
    _logit = tf.add(tf.matmul(_dense, _w['dense_w']), _b['dense_b'])
    _out = {
        'input_r': _input_r, 'conv': _conv, 'biase': _bias, 'relu': _relu
        , 'pool': _pool, 'dense': _dense, 'logit': _logit
    }
    return _out

# PREDICTION
cnn_out = CNN(x, weights, biases)

# LOSS AND OPTIMIZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        labels=y, logits=cnn_out['logit']))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 
corr = tf.equal(tf.argmax(cnn_out['logit'], 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# INITIALIZER
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")

FUNCTIONS READY


In [17]:
saver_cw = tf.train.Saver([weights['conv_w']])
print("conv_weigt")
print(weights['conv_w'].shape)

saver_cb = tf.train.Saver([biases['conv_b']])
print("conv_bias")
print(biases['conv_b'].shape)

saver_dw = tf.train.Saver([weights['dense_w']])
print("dense_weight")
print(weights['dense_w'].shape)

saver_db = tf.train.Saver([biases['dense_b']])
print("dense_bias")
print(biases['dense_b'].shape)

print("Save Ready.")

conv_weigt
(3, 3, 1, 64)
conv_bias
(64,)
dense_weight
(262144, 2)
dense_bias
(2,)
Save Ready.


In [18]:
# PARAMETERS
training_epochs = 25
batch_size      = 50
display_step    = 5
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)

# OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(train_cnt/batch_size)
    # ITERATION
    for i in range(total_batch):
        randidx = np.random.choice(trainimg.shape[0], size=batch_size)
        batch_xs = trainimg[randidx,:]
        batch_ys = trainlabel[randidx,:]
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch+1, training_epochs, avg_cost))
        feeds = {x: trainimg, y: trainlabel}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: testimg, y: testlabel}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
    
print ("OPTIMIZATION FINISHED")

Epoch: 005/025 cost: 0.873946196
TRAIN ACCURACY: 0.636
TEST ACCURACY: 0.640
Epoch: 010/025 cost: 0.263016914
TRAIN ACCURACY: 0.913
TEST ACCURACY: 0.750
Epoch: 015/025 cost: 0.167516857
TRAIN ACCURACY: 0.983
TEST ACCURACY: 0.740
Epoch: 020/025 cost: 0.071072880
TRAIN ACCURACY: 0.995
TEST ACCURACY: 0.770
Epoch: 025/025 cost: 0.043495178
TRAIN ACCURACY: 0.996
TEST ACCURACY: 0.790
OPTIMIZATION FINISHED


In [19]:
print(weights['conv_w'].shape)
save_path = saver_cw.save(sess, "CNN_Save_128/conv_w.ckpt")

print(biases['conv_b'].shape)
save_path = saver_cb.save(sess, "CNN_Save_128/conv_b.ckpt")

print(weights['dense_w'].shape)
save_path = saver_dw.save(sess, "CNN_Save_128/dense_w.ckpt")

print(biases['dense_b'].shape)
save_path = saver_db.save(sess, "CNN_Save_128/dense_b.ckpt")

print("Model saved in file: %s" % save_path)

(3, 3, 1, 64)
(64,)
(262144, 2)
(2,)
Model saved in file: CNN_Save_128/dense_b.ckpt
